## 부천시 버스


In [92]:
import pandas as pd
import re

In [86]:
# 파일 불러오기
bus_data = pd.read_csv('data/경기도 부천시_버스정보안내기설치현황_20200130.csv', encoding='euc-kr')

# 비어있는 columns 제거
bus_data = bus_data.iloc[:, [0,1,2,3,4,5,6]]
bus_data

# index -> columns['번호'] 변경
bus_data.set_index('번호', inplace=True)
bus_data

# 비어있는 rows 제거
bus_data = bus_data.dropna(axis=0, thresh=1)
bus_data

# columns의 NaN 개수 확인
# nan_bus_data = bus_data.isnull().sum(axis=0)
# print(nan_bus_data)

# columns의 NaN 삭제


,지역,행정동,정류소ID,정류소 명칭,안내기유형,세부유형
번호,,,,,,
1.0,오정,도당동,11001.0,장미공원앞,LED,표지판일체형
2.0,원미,상동,11002.0,삼산체육관역.영상문화단지정문,LED,쉘터(거치형)
3.0,원미,상동,11005.0,벚꽃마을,LED,쉘터(거치형)
4.0,원미,상동,11006.0,벚꽃마을,LCD,쉘터(독립형)
5.0,원미,상동,11008.0,상동고,LCD,독립형
...,...,...,...,...,...,...
813.0,오정,고강동,13356.0,백산빌라,LED,독립형
814.0,소사,옥길동,12465.0,옥길브리즈힐후문,LED,쉘터(거치형)
815.0,소사,옥길동,12192.0,스타필드시티부천,LCD+LED,쉘터(거치형)


In [91]:
# 파일 불러오기
bus_stops = pd.read_csv('data/버스정류소현황.csv')
bus_stops
# bucheon_filter = bus_stops['시군명'].isin(['부천시'])
# bucheon_filter


,시군명,정류소명,정류소영문명,정류소id,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도
0,가평군,3반상회앞,3-ban Market,239000307,44655.0,노변정류장,경기도 가평군,경기도 가평군 가평읍,37.811250,127.521467
1,가평군,가는골,Ganeungol,239000874,44847.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.967533,127.418617
2,가평군,가는골,Ganeungol,239000875,44848.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.967450,127.418650
3,가평군,가둘기,Gadulgi,239000061,44280.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.900067,127.511583
4,가평군,가둘기(목동방면),Gadulgi,239000459,44354.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.900033,127.511700
...,...,...,...,...,...,...,...,...,...,...
34580,화성시,흥법리마을회관,Heungbeop-ri Community Center,233002602,NaN,노변정류장,경기도 화성시,NaN,37.164950,126.734033
34581,화성시,흥법리모래지,Heungbeop-ri Moraeji,233002916,NaN,노변정류장,경기도 화성시,NaN,37.166583,126.740150
34582,화성시,흥법리모래지,Heungbeop-ri Moraeji,233002603,NaN,노변정류장,경기도 화성시,NaN,37.166583,126.739967
34583,화성시,흥법사입구,Heungbeopsa,233000574,36217.0,노변정류장,경기도 화성시,경기도 화성시 봉담읍,37.200050,126.980350
